In [0]:
spark.conf.set(
  "fs.azure.account.key.goodreadsreviews60300832.dfs.core.windows.net",
  "REMOVED_SECRET=="
)

In [0]:
# ==== 0) CONFIG ====
storage_acct = "goodreadsreviews60300832"   
container    = "lakehouse"

gold_path  = f"abfss://{container}@{storage_acct}.dfs.core.windows.net/gold"
train_path = f"{gold_path}/features_v2/train"   # note: plural "features_v2"


In [0]:
%pip install emoji==2.14.0 nltk==3.9.1

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
import re

# If nltk was installed via %pip, ensure VADER lexicon is available
try:    import nltk
except ModuleNotFoundError:
    pass

try:
    nltk.data.find("sentiment/vader_lexicon.zip")
except LookupError:
    nltk.download("vader_lexicon")

In [0]:
%pip install vaderSentiment==3.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/126.0 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# ============ 1) LOAD ============
def read_delta_or_parquet(path: str):
    try:
        return spark.read.format("delta").load(path)
    except Exception:
        return spark.read.format("parquet").load(path)

train = read_delta_or_parquet(train_path)
# Expect at least: review_id, review_text (others will pass-through if present)
train.printSchema()
display(train.select("review_id","review_text").limit(10))

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- review_length_chars: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- book_review_count: long (nullable = true)
 |-- book_avg_rating: double (nullable = true)



review_id,review_text
000197dca1e705ed3a3e7be5494e41c7,"this has to be one of my all time favourite reads, i think it such a passionate and a haunting book. an old teacher and i would spend ages arguing about it as i thought it was so romantic and he would disagree but i stand by it! it has one of the most beautiful lines i've ever read 'i love everything about her, the air above her head and the ground beneath her feet...(or words to that effect) heathcliff and cathy loved each other so much it destroyed them, they loved each other in this world and the next....ok when you put it like that he had a point! a great book with some legendary characters."
000250b46f177fe76441bbd178a64671,amazing graphics. and very cool to read a part of the story i never knew about before. also made more palatable by the lack of king's involvement. i'm still pissed at him.
00027dd587d48a1ea0799f140b0fef60,una biografia escrita como si fuera un bonito paseo por la vida del autor. una manera de ver a traves de los ojos del otro el comienzo dr una etapa y el fin de muchas otras. me gusto el libro.
00064c6ed6be9052c383a5f8a26d1cf9,"susanna kearsley knocks us off our feet with her spectacular tale of love overcoming all obstacles in her wondrous time travel romance, the rose garden . mourning the death of her beloved sister, eva returns to cornwall to scatter her ashes at the estate where they spent their happiest childhood summers. the people and the countryside are as she remembers--but not quite. voices speak from empty rooms, paths appear in places where none exist, and a handsome man dressed in breeches and boots beckons her to another time. daniel doesn't know what to make of the beautiful woman wearing strange clothes who appears on the hillside beside him. 1715 england is a dangerous place, with the jacobites threatening the reign of the newly crowned george i. is she involved in the upcoming confrontation between the supporters of george i and those of james stuart, whom some consider the rightful king? the rose garden is a masterwork of suspenseful romance. with vivid descriptions of the unchanging atlantic and its salt spray, the green of the hilly cornwall estate's rose gardens, and faithful renditions of the tensions and way of life of two very different eras, ms. kearsley melds past and present into a coherent whole that firmly anchors the reader into each time period. we live eva's confusion, disbelief and terror as she fades in and out of a past where danger lurks at every turn. both eva and daniel are people of their eras, but unique enough to accept something as outlandish as time travel, and to love a person of extraordinary background. daniel is as forceful as any eighteenth century man, but he also possesses the sensitivity a modern woman appreciates. and while eva has plenty of twenty-first century backbone, she is smart enough to adapt to the harsh conditions in 1715 without betraying herself. the romance is breath-taking, as eva and daniel, stronger together than singly, seek to triumph over villains and situations that would have destroyed lesser souls. and ms. kearsley leaves us gasping at the end with a corker of a surprise. read the rose garden . you'll be happy you did. arc provided by sourcebooks"
000671601b00cc04bd8eaf95de8f1ab4,"my copy, provided by the author in exchange for an honest review, is for the second edition from rocky ridge books. nice presentation, btw. this second book in the dark angels series is an excellent second look at the band and all its gay members. josh and robin showed up in the first book as supporting characters to larger than life angel and his waffling bisexual love interest dare. we could see clearly from their brief moments in the first book that there are issues. a huge theme is ""who is family?"" and ""what does that mean?"" the band's prurient audiences love the idea of twincest, (which for me is automatic back button) but the guys are stepbrothers. it's a close enough connection to keep

In [0]:
# ============ 2) TEXT CLEANING & NORMALIZATION ============
# Rules:
# - lowercase
# - replace URLs -> <URL>, numbers -> <NUM>, emojis -> <EMOJI>
# - remove punctuation
# - collapse extra spaces
# - trim
# - drop very short (<10 chars)

url_re   = re.compile(r"(https?://\S+|www\.\S+)", flags=re.IGNORECASE)
num_re   = re.compile(r"\b\d+([.,]\d+)*\b")
punc_re  = re.compile(r"[^\w\s]", flags=re.UNICODE)
space_re = re.compile(r"\s+")

# Broad emoji ranges (no external dependency required)
emoji_re = re.compile(
    "[\U0001F300-\U0001F6FF\U0001F900-\U0001FAFF\U00002700-\U000027BF\U00002600-\U000026FF]"
)

@F.udf(returnType=T.StringType())
def clean_text_u(text):
    if text is None:
        return None
    s = text.strip().lower()
    s = url_re.sub(" <URL> ", s)
    s = num_re.sub(" <NUM> ", s)
    s = emoji_re.sub(" <EMOJI> ", s)
    s = punc_re.sub(" ", s)          # remove punctuation but keep word boundaries
    s = space_re.sub(" ", s).strip() # collapse spaces
    return s

cleaned = (
    train
    .withColumn("clean_text", clean_text_u(F.col("review_text")))
    .withColumn("_len_chars", F.length("clean_text"))
    .filter(F.col("_len_chars") >= 10)   # drop very short
    .drop("_len_chars")
)

display(cleaned.select("review_id","clean_text").limit(10))

review_id,clean_text
000118172d28cf5a070fe4c3c5272959,this extraordinary novel centering on lilith born into slavery on a jamaica plantation in the late 18th century deals with the dehumanizing effect of slavery on both the slaveholders and the enslaved it s moral complex and emotionally challenging to read the characters are damaged and very very real james writes in dialect effectively by capturing the rhythm and word usage highest recommendation
0001eb3b3b31e2c5b7853b2449114f99,i voluntarily received an advance reader copy of this book this story definitely takes a reader for a walk on the dark side of romance there is so much forbidden about this story that it is impossible to put it down you just have to know what is going to happen next his muse is one may december romance that is not for the faint of heart but the happily ever after made all the drama and suspense worth while
00048c0e175f115459d66d4bc26bf9a3,this was a super quick read which is actually beneficial if you re interested in a brief overview regarding the maori and the british crown esp the treaty of waitangi then this book has a lot of simple and quick information to pick up you won t end up being an expert but you can learn what you need here the writing is not only informative but is actually quite upbeat and it was a pleasant read no droll scholarship here great writing good information and something i would recommend
00072889d415fffaa40ebb31c46163cc,i so thought i would of given this NUM stars just because there was not enough of edward in this book but i couldn t help myself it gets NUM awesome stars i m officially addicted to edward and bella i so want her to become part of the cullen family NUM its a shame these books sat on my bookshelf for so many years
0007d67628b02f7731c73e6f683dbe08,i had no sooner finished reading embattled minds the book i last reviewed when i started reading this one i was so taken with the first book by this author by her writing by her realness by the characters by the concept that i had no choice but to read the first book in the series this is the story of john palmer and shannon murphy john is one of founders of lost n found investigative agency a detective agency organized and run by wounded vets john is quite the gruff unfriendly kind of guy he was paralyzed from an injury but yes he can wink and has become even more unsociable since being stuck in a wheel chair his past is a sad and lonely one so the reader can see why he is the way he is shannon has recently been hired by the agency as the office manager from the beginning they notice each other but because each has quite a set of baggage they don t act on any attraction they may have that is until john notices that shannon seems off and wants to know why it turns out that something has gone terribly wrong she had a very scary stalker years ago but he is in prison now yet many of the same things are starting to happen that he did years ago a pepsi can left where she can t help but see it and it has the stalkers finger prints yet he is in jail john takes no time in moving in with her to protect her and get the rest of agency on the case this one didn t get to me quite as much as embattled minds while the previous is a NUM this one would be a NUM NUM NUM if we could give NUM NUM marks i love a gruff teddy bear type of hero but i didn t find john teddy bearish he is reluctant to believe that shannon could really be interested in him at all he was often rude and while i could understand why he is the way he is he just doesn t have that inner marshmallow that makes this kind of hero so attractive i like shannon though she isn t a real stand out kind of heroine i did appreciate how calm and collected she remained despite the threat she is under and i also liked how she wasn t bothered at all by john s infirmity she wants him she goes after him that simple like the first one i read embattled hearts this one is also novella size in length but again like the first one i foun

In [0]:
# ============ 3) BASIC TEXT FEATURES ============
with_basic = (
    cleaned
    .withColumn("review_length_chars", F.length("clean_text").cast("int"))
    .withColumn("review_length_words",
                F.size(F.split(F.col("clean_text"), r"\s+")).cast("int"))
)

display(with_basic.select("review_id","review_length_words","review_length_chars").limit(10))

review_id,review_length_words,review_length_chars
000118172d28cf5a070fe4c3c5272959,63,398
0001eb3b3b31e2c5b7853b2449114f99,80,409
00048c0e175f115459d66d4bc26bf9a3,90,485
00072889d415fffaa40ebb31c46163cc,65,314
0007d67628b02f7731c73e6f683dbe08,551,2720
000819bbe58c374dfb956abe6238f30c,220,1169
00085ae1bfa285138f710e5009c79381,45,214
000aa569b4f3aeb75312edd7f7d3c8f2,253,1386
000c29d5198b932bfd167ce9ca31c611,39,200
000dd791f3d4868b490d65e69b0eaa85,54,309


In [0]:
from pyspark.sql import functions as F, types as T

# ============ 4) SENTIMENT (vaderSentiment) ============
@F.udf(returnType=T.StructType([
    T.StructField("pos", T.DoubleType(), True),
    T.StructField("neu", T.DoubleType(), True),
    T.StructField("neg", T.DoubleType(), True),
    T.StructField("compound", T.DoubleType(), True),
]))
def vader_scores_u(text):
    if text is None or text == "":
        return (0.0, 1.0, 0.0, 0.0)
    # No external data files needed
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()
    s = sia.polarity_scores(text)
    return (float(s["pos"]), float(s["neu"]), float(s["neg"]), float(s["compound"]))

with_sent = (
    with_basic
    .withColumn("vader", vader_scores_u(F.col("clean_text")))
    .withColumn("sentiment_pos", F.col("vader.pos"))
    .withColumn("sentiment_neu", F.col("vader.neu"))
    .withColumn("sentiment_neg", F.col("vader.neg"))
    .withColumn("sentiment_compound", F.col("vader.compound"))
    .drop("vader")
)

display(with_sent.select("review_id","sentiment_pos","sentiment_neu","sentiment_neg","sentiment_compound").limit(10))


review_id,sentiment_pos,sentiment_neu,sentiment_neg,sentiment_compound
000118172d28cf5a070fe4c3c5272959,0.085,0.684,0.231,-0.93
0001eb3b3b31e2c5b7853b2449114f99,0.178,0.799,0.022,0.9226
00048c0e175f115459d66d4bc26bf9a3,0.235,0.739,0.026,0.9719
00072889d415fffaa40ebb31c46163cc,0.148,0.798,0.054,0.7999
0007d67628b02f7731c73e6f683dbe08,0.132,0.774,0.094,0.9814
000819bbe58c374dfb956abe6238f30c,0.097,0.76,0.142,-0.8979
00085ae1bfa285138f710e5009c79381,0.226,0.774,0.0,0.9001
000aa569b4f3aeb75312edd7f7d3c8f2,0.126,0.845,0.029,0.9666
000c29d5198b932bfd167ce9ca31c611,0.157,0.843,0.0,0.7845
000dd791f3d4868b490d65e69b0eaa85,0.3,0.656,0.044,0.9559


In [0]:
# =================== 5) TF–IDF in pure Spark (no MLlib, no broadcasts) ===================
from pyspark.sql import functions as F, types as T
from pyspark.ml.linalg import Vectors, VectorUDT
import math, numpy as np

# 5.1 Tokenize via SQL split (whitespace)
df_tok = with_sent.withColumn("tokens", F.split(F.col("clean_text"), r"\s+"))

# 5.2 Stopword removal via UDF (STOPWORDS captured as a constant, not broadcast)
STOPWORDS = {
    "a","about","above","after","again","against","all","am","an","and","any","are","as","at",
    "be","because","been","before","being","below","between","both","but","by",
    "can","could",
    "did","do","does","doing","down","during",
    "each","few","for","from","further",
    "had","has","have","having","he","her","here","hers","herself","him","himself","his","how",
    "i","if","in","into","is","it","its","itself",
    "just",
    "me","more","most","my","myself",
    "no","nor","not","now",
    "of","off","on","once","only","or","other","our","ours","ourselves","out","over","own",
    "same","she","should","so","some","such",
    "than","that","the","their","theirs","them","themselves","then","there","these","they",
    "this","those","through","to","too",
    "under","until","up",
    "very",
    "was","we","were","what","when","where","which","while","who","whom","why","will","with",
    "you","your","yours","yourself","yourselves"
}

@F.udf(returnType=T.ArrayType(T.StringType()))
def remove_stop_u(tokens):
    if tokens is None:
        return []
    return [t for t in tokens if t and (t not in STOPWORDS)]

df_sw = df_tok.withColumn("tokens_sw", remove_stop_u(F.col("tokens")))

# 5.3 Bigrams in SQL (adjacent pairs)
@F.udf(returnType=T.ArrayType(T.StringType()))
def bigrams_u(tokens):
    if not tokens:
        return []
    n = len(tokens)
    if n < 2:
        return []
    return [tokens[i] + " " + tokens[i+1] for i in range(n-1)]

df_bg = df_sw.withColumn("bigrams", bigrams_u(F.col("tokens_sw")))

# 5.4 Combine unigrams + bigrams
df_grams = df_bg.withColumn("tokens_all", F.concat(F.col("tokens_sw"), F.col("bigrams")))

# 5.5 Document frequency (DF) per term in Spark; filter minDF>=5
expl = (df_grams
        .select("review_id", F.explode_outer("tokens_all").alias("term"))
        .dropna(subset=["term"])
        .dropDuplicates(["review_id","term"])
       )

N_docs = df_grams.select(F.countDistinct("review_id").alias("N")).collect()[0]["N"]
df_df = (expl.groupBy("term")
              .agg(F.count("*").alias("df"))
              .filter(F.col("df") >= 5))

# 5.6 Vocabulary (top DF terms up to MAX_FEATURES)
MAX_FEATURES = 20000
vocab_rows = (df_df.orderBy(F.desc("df"))
                   .limit(MAX_FEATURES)
                   .collect())

# Build global vocab + idf **without** broadcast; UDFs will capture these globals
VOCAB_LIST  = [r["term"] for r in vocab_rows]
VOCAB_INDEX = {t: i for i, t in enumerate(VOCAB_LIST)}

IDF_VEC = np.zeros(len(VOCAB_LIST), dtype="float32")
df_map  = {r["term"]: r["df"] for r in vocab_rows}
for term, idx in VOCAB_INDEX.items():
    df_term = df_map.get(term, 0)
    # sklearn-style idf
    IDF_VEC[idx] = math.log((N_docs + 1.0) / (df_term + 1.0)) + 1.0
IDF_VEC = IDF_VEC.tolist()  # make it JSON-serializable for the UDF

# 5.7 Build TF–IDF SparseVector via scalar UDF (captures VOCAB_INDEX & IDF_VEC)
@F.udf(returnType=VectorUDT())
def tfidf_from_tokens(tokens_all):
    vocab = VOCAB_INDEX
    idf   = IDF_VEC
    if not tokens_all:
        return Vectors.sparse(len(idf), [], [])
    tf = {}
    for t in tokens_all:
        idx = vocab.get(t)
        if idx is not None:
            tf[idx] = tf.get(idx, 0) + 1
    if not tf:
        return Vectors.sparse(len(idf), [], [])
    idxs = sorted(tf.keys())
    vals = [tf[i] * idf[i] for i in idxs]
    # L2 normalize
    norm = math.sqrt(sum(v*v for v in vals))
    if norm > 0:
        vals = [v / norm for v in vals]
    return Vectors.sparse(len(idf), idxs, vals)

with_tfidf = df_grams.withColumn("tfidf_features", tfidf_from_tokens(F.col("tokens_all")))
with_tfidf.select("review_id","tfidf_features").limit(1).printSchema()


root
 |-- review_id: string (nullable = true)
 |-- tfidf_features: vectorudt (nullable = true)



In [0]:
# --- 1) Inspect column list and schema ---
print("Column names →")
print(with_tfidf.columns)
print("\nSchema →")
with_tfidf.printSchema()

# --- 2) Show a sample of 10 rows with selected features ---
display(
    with_tfidf.select(
        "review_id",
        "clean_text",
        "review_length_words",
        "sentiment_compound",
        "tfidf_features"
    ).limit(10)
)


Column names →
['review_id', 'user_id', 'book_id', 'author_id', 'author_name', 'title', 'language_code', 'n_votes', 'rating', 'review_text', 'review_length_words', 'review_length_chars', 'date_added', 'book_review_count', 'book_avg_rating', 'clean_text', 'sentiment_pos', 'sentiment_neu', 'sentiment_neg', 'sentiment_compound', 'tokens', 'tokens_sw', 'bigrams', 'tokens_all', 'tfidf_features']

Schema →
root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_length_words: integer (nullable = false)
 |-- review_length_chars: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- book_review_count: long (nullable = tr

review_id clean_text review_length_words sentiment_compound tfidf_features 000118172d28cf5a070fe4c3c5272959 this extraordinary novel centering on lilith born into slavery on a jamaica plantation in the late 18th century deals with the dehumanizing effect of slavery on both the slaveholders and the enslaved it s moral complex and emotionally challenging to read the characters are damaged and very very real james writes in dialect effectively by capturing the rhythm and word usage highest recommendation 63 -0.93 Map(vectorType -> sparse, length -> 20000, indices -> List(1, 3, 10, 61, 113, 465, 610, 715, 759, 993, 1172, 1175, 1320, 1650, 1679, 1892, 2967, 3524, 3698, 3939, 4754, 5802, 5869, 5921, 6908, 7532, 9904, 13598, 14013, 15794), values -> List(0.04932970706592388, 0.05186490165864906, 0.06914555506243575, 0.09223831842048257, 0.10214211420969792, 0.13328630462895785, 0.13963342520552474, 0.14374470644837106, 0.14510381759293095, 0.15208747605810158, 0.1569790431498257, 0.15702907566019236, 0.16055810652765798, 0.16681385651657263, 0.16711755569877193, 0.17074422768441494, 0.18308293239144924, 0.1877963162985448, 0.18923448091722536, 0.19098427509494584, 0.39277538800172146, 0.20204040601530082, 0.20248570589627082, 0.20277781250102866, 0.20721355417840126, 0.20962266302472127, 0.21778038611704248, 0.22663928889058702, 0.22750580770756912, 0.23095475757700448)) 0001eb3b3b31e2c5b7853b2449114f99 i voluntarily received an advance reader copy of this book this story definitely takes a reader for a walk on the dark side of romance there is so much forbidden about this story that it is impossible to put it down you just have to know what is going to happen next his muse is one may december romance that is not for the faint of heart but the happily ever after made all the drama and suspense worth while 80 0.9226 Map(vectorType -> sparse, length -> 20000, indices -> List(0, 4, 5, 13, 29, 47, 51, 67, 94, 100, 106, 107, 138, 142, 154, 198, 229, 257, 276, 277, 284, 304, 519, 593, 804, 1266, 1388, 1468, 1817, 2226, 2311, 2457, 2931, 3508, 3617, 3960, 6006, 6205, 6518, 7534, 8856, 10784, 11553, 12045, 12869, 14283, 16926), values -> List(0.039120737265257575, 0.10500280686251472, 0.05294648394851563, 0.06575914674703631, 0.0738249485620122, 0.0809469650399376, 0.08174083630948559, 0.08440352407252882, 0.09085893149754987, 0.09142831964220927, 0.1839491449155402, 0.09204372178377809, 0.09721125061791108, 0.19494153004828224, 0.09835785806093922, 0.10313603172158697, 0.10648362311536926, 0.10931740248658622, 0.11020471040260196, 0.1102545743188273, 0.11098435189284886, 0.11229978277453127, 0.12305020197048336, 0.1256269499842074, 0.13276754157672382, 0.14430271161320876, 0.1467265133749148, 0.1480231662005229, 0.15354226727904052, 0.15882051263005934, 0.15978695578683585, 0.1613968308853481, 0.1655830242391491, 0.1700849229145914, 0.17081412747736593, 0.1731453879226777, 0.18411886368426797, 0.18487800808469945, 0.1861973067922993, 0.18996411560195073, 0.1942538688009169, 0.19943101348003903, 0.20121244575836322, 0.2022952937624257, 0.20392734917129146, 0.20668730559351692, 0.21106047967986227)) 00048c0e175f115459d66d4bc26bf9a3 this was a super quick read which is actually beneficial if you re interested in a brief overview regarding the maori and the british crown esp the treaty of waitangi then this book has a lot of simple and quick information to pick up you won t end up being an expert but you can learn what you need here the writing is not only informative but is actually quite upbeat and it was a pleasant read no droll scholarship here great writing good information and something i would recommend 90 0.9719 Map(vectorType -> sparse, length -> 20000, indices -> List(0, 2, 3, 11, 12, 24, 33, 56, 59, 64, 98, 109, 112, 122, 143, 246, 314, 315, 329, 372, 434, 507, 514, 597, 764, 918, 1585, 1712, 2160, 2161, 2406, 2767, 4365, 4876, 5003, 5169, 5607, 6299, 6542, 9606, 10050, 11159, 15420, 15823, 18997, 19377, 19837), values -> List(0.0371

In [0]:
# 1) Drop intermediate text-processing columns
final_tfidf = (
    with_tfidf
    .drop("tokens", "tokens_sw", "bigrams", "tokens_all")  # intermediate arrays
)

# 2) Verify schema before saving
print("Final columns →", final_tfidf.columns)
final_tfidf.printSchema()


Final columns → ['review_id', 'user_id', 'book_id', 'author_id', 'author_name', 'title', 'language_code', 'n_votes', 'rating', 'review_text', 'review_length_words', 'review_length_chars', 'date_added', 'book_review_count', 'book_avg_rating', 'clean_text', 'sentiment_pos', 'sentiment_neu', 'sentiment_neg', 'sentiment_compound', 'tfidf_features']
root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_length_words: integer (nullable = false)
 |-- review_length_chars: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- book_review_count: long (nullable = true)
 |-- book_avg_rating: double (nullable = true)
 |-- c

In [0]:
%pip install -q sentence-transformers

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql import functions as F, types as T
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.functions import vector_to_array

In [0]:
# ===========================================
# SBERT embeddings with low-memory settings
# ===========================================

# %pip install -q sentence-transformers

from typing import Iterator
import os, errno
import pandas as pd

from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType
from pyspark.sql.functions import pandas_udf, PandasUDFType

# ---- 0) Conservative Spark/Arrow knobs ----
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "128")   # ↓ Arrow batch → ↓ RAM spikes
spark.conf.set("spark.sql.shuffle.partitions", "200")                   # moderate shuffle size

# ---- 1) Pick a writable local cache on workers ----
def _pick_cache_dir():
    for d in ["/tmp/hf_cache", "/var/tmp/hf_cache"]:
        try:
            os.makedirs(d, exist_ok=True)
            testfile = os.path.join(d, ".touch")
            with open(testfile, "w") as f:
                f.write("ok")
            os.remove(testfile)
            return d
        except OSError as e:
            if e.errno not in (errno.EACCES, errno.EROFS):
                raise
    raise OSError("No writable cache dir available (tried /tmp and /var/tmp).")

# ---- 2) Low-memory SBERT iterator UDF (keeps Array<Float>) ----
@pandas_udf(ArrayType(FloatType()), PandasUDFType.SCALAR_ITER)
def sbert_embed_iter(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    cache_dir = _pick_cache_dir()
    os.environ["HF_HOME"] = cache_dir
    os.environ["TRANSFORMERS_CACHE"] = cache_dir
    os.environ["SENTENCE_TRANSFORMERS_HOME"] = cache_dir
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    global _sbert_model
    if "_sbert_model" not in globals():
        from sentence_transformers import SentenceTransformer
        _sbert_model = SentenceTransformer(
            "sentence-transformers/all-MiniLM-L6-v2",
            device="cpu",
            cache_folder=cache_dir,
        )
        # ↓ truncate long reviews to reduce token memory/compute
        _sbert_model.max_seq_length = 256

    for s in batch_iter:
        arr = s.fillna("").astype(str)
        embs = _sbert_model.encode(
            arr.tolist(),
            batch_size=32,                   # CRITICAL: smaller = less RAM
            normalize_embeddings=True,
            show_progress_bar=False,
        )
        yield pd.Series([list(map(float, e)) for e in embs])

# ---- 3) Minimal input DF (only what we need) ----
text_df = with_sent.select("review_id", "clean_text").repartition(200)
text_df = text_df.persist()

# ---- 4) Sharded compute+write to avoid one giant action ----
emb_out = f"{gold_path}/features_v2/train_text_embeddings_sbert"
# Clean target once
spark.sql(f"DROP TABLE IF EXISTS delta.`{emb_out}`")  # safe even if path table doesn't exist

SHARDS = 50  # increase if still OOM; 50–100 is typical
for i in range(SHARDS):
    part = (text_df
            .where(F.pmod(F.hash("review_id"), F.lit(SHARDS)) == F.lit(i))
            .withColumn("bert_embedding", sbert_embed_iter(F.col("clean_text")))
            .select("review_id", "bert_embedding"))
    (part.write.format("delta")
         .mode("append")          # append shard-by-shard
         .save(emb_out))

text_df.unpersist()

# ---- 5) Load embeddings and join with your TF-IDF features ----
emb_df = spark.read.format("delta").load(emb_out)
# Optionally: small sanity check (dimension)
emb_df.select(F.size("bert_embedding").alias("dim")).limit(5).show()

train_all = (
    final_tfidf
    .join(emb_df, on="review_id", how="inner")
)

train_all_out = f"{gold_path}/features_v2/train_allfeatures"
(train_all.write.format("delta").mode("overwrite").save(train_all_out))

print(train_all_out)
train_all.printSchema()

/databricks/spark/python/pyspark/sql/pandas/functions.py:443: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


stuff that failed. Plans that i shared and the haters indeed sabotaged me


In [0]:

# ==========================================
# SAVE TF-IDF OUTPUT (TEMP ONLY)
# ==========================================

tmp = f"{gold_path}/features_v2/_tmp_train"

# clean stale tmp path if any
dbutils.fs.rm(tmp, recurse=True)

spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

(final_tfidf
  .repartition(96)
  .write
  .format("delta")
  .mode("overwrite")
  .option("overwriteSchema", "true")
  .option("maxRecordsPerFile", 5000)
  .save(tmp))

print(f"✅ TF-IDF data written to temporary path:\n{tmp}")

# verify it committed
display(dbutils.fs.ls(f"{tmp}/_delta_log"))
ver = spark.read.format("delta").load(tmp)
print("rows →", ver.count())
ver.printSchema()
display(ver.limit(10))


✅ TF-IDF data written to temporary path:
abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v2/_tmp_train


path,name,size,modificationTime
abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v2/_tmp_train/_delta_log/00000000000000000000.crc,00000000000000000000.crc,4435,1762831562000
abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v2/_tmp_train/_delta_log/00000000000000000000.json,00000000000000000000.json,3985133,1762831561000
abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v2/_tmp_train/_delta_log/00000000000000000001.crc,00000000000000000001.crc,4450,1762831578000
abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v2/_tmp_train/_delta_log/00000000000000000001.json,00000000000000000001.json,277842,1762831577000
abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v2/_tmp_train/_delta_log/00000000000000000002.crc,00000000000000000002.crc,4452,1762831589000
abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v2/_tmp_train/_delta_log/00000000000000000002.json,00000000000000000002.json,280059,1762831588000
abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v2/_tmp_train/_delta_log/__tmp_path_dir/,__tmp_path_dir/,0,1762831560000
abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v2/_tmp_train/_delta_log/_staged_commits/,_staged_commits/,0,1762813986000


rows → 9491109
root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- review_length_chars: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- book_review_count: long (nullable = true)
 |-- book_avg_rating: double (nullable = true)
 |-- clean_text: string (nullable = true)
 |-- sentiment_pos: double (nullable = true)
 |-- sentiment_neu: double (nullable = true)
 |-- sentiment_neg: double (nullable = true)
 |-- sentiment_compound: double (nullable = true)
 |-- tfidf_features: vectorudt (nullable = true)



review_id,user_id,book_id,author_id,author_name,title,language_code,n_votes,rating,review_text,review_length_words,review_length_chars,date_added,book_review_count,book_avg_rating,clean_text,sentiment_pos,sentiment_neu,sentiment_neg,sentiment_compound,tfidf_features
6a39fd38d369346e0d143d0099d96f08,eab84eec25f0050ea4c575a6d43a5619,25817531,8183303,Gilly Macmillan,What She Knew (Jim Clemo #1),eng,0,4.0,"this was a pretty compelling book. macmillan does a great job of offering up a variety of plausible suspects which kept me reading and trying to figure out who the actual bad guy was before they were revealed in the book. i wasn't successful in my guessing which to me makes for a great suspense novel. it is told in alternating perspectives between the mother, rachel and the lead detective which made it more interesting because you get to see all sides of the case. also, as a bonus, i listened to this on audio and it is a british narrator, which i just love :) it made it even better to listen to. in a nutshell: a well told story that kept me not only guessing but (metaphorically) turning the pages to find out if ben would be found and if he would be found alive.",147,755,2016-04-11,404,3.8193069306930694,this was a pretty compelling book macmillan does a great job of offering up a variety of plausible suspects which kept me reading and trying to figure out who the actual bad guy was before they were revealed in the book i wasn t successful in my guessing which to me makes for a great suspense novel it is told in alternating perspectives between the mother rachel and the lead detective which made it more interesting because you get to see all sides of the case also as a bonus i listened to this on audio and it is a british narrator which i just love it made it even better to listen to in a nutshell a well told story that kept me not only guessing but metaphorically turning the pages to find out if ben would be found and if he would be found alive,0.154,0.822,0.024,0.9423,"Map(vectorType -> sparse, length -> 20000, indices -> List(0, 2, 4, 9, 12, 16, 17, 18, 23, 24, 25, 36, 49, 51, 52, 61, 62, 76, 83, 84, 97, 121, 128, 156, 180, 205, 212, 231, 235, 324, 347, 519, 547, 656, 750, 770, 801, 927, 981, 1150, 1199, 1233, 1333, 1344, 1421, 1487, 1564, 1585, 1798, 1956, 2078, 2130, 2320, 2323, 2334, 2527, 2611, 2770, 3025, 3343, 3392, 4226, 4241, 4539, 4731, 5263, 6025, 6447, 6991, 7384, 10073, 10101, 12450, 16116, 16271, 18702), values -> List(0.0609471763396232, 0.03656680307767698, 0.04089662563497029, 0.04843150444351891, 0.10222035760930642, 0.05255712921391389, 0.053003576490610875, 0.053920536979270246, 0.0553089173370359, 0.11084731657147501, 0.05582779954524364, 0.05948746298749091, 0.0636015011445394, 0.12734609603400307, 0.06375779488757387, 0.06511072262625167, 0.13024066638395373, 0.06818110077068752, 0.07008234685284456, 0.070085122618562, 0.07096507754717939, 0.07396073784391143, 0.07486710206779687, 0.07708523644555153, 0.0791103737392777, 0.08088120997747726, 0.16307648697358823, 0.1660479976436262, 0.08328652023544428, 0.08826629972680833, 0.0893681903756775, 0.09585149568208069, 0.09668274684873807, 0.09991472384288107, 0.1021771170378288, 0.10272904341178832, 0.10340331735541045, 0.10578381775430169, 0.10716270843396135, 0.11037907232713424, 0.11124973285764585, 0.11189723484314237, 0.11352833429261197, 0.11368456759183684, 0.22954016523566062, 0.1155344178377451, 0.11669558061620534, 0.11696267716127028, 0.1193520581506478, 0.1212071437603715, 0.12233795444736874, 0.1228920010041878, 0.12455451263655018, 0.12460711734899081, 0.12465625351666561, 0.1261993560784163, 0.12686117859726243, 0.12797599492233674, 0.12963615389564884, 0.13164629182554846, 0.13188153913248205, 0.1362766967984209, 0.1363702917128128, 0.13768339163049303, 0.13854550633623305, 0.14061093640333827, 0.1434796739989739, 0.14479071882712802, 0.1464955225281098, 0.14759076435792384, 0.15412528881867085, 0.1541662418244316, 0.15822593634713272, 0.163441

In [0]:
ver.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- review_length_chars: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- book_review_count: long (nullable = true)
 |-- book_avg_rating: double (nullable = true)
 |-- clean_text: string (nullable = true)
 |-- sentiment_pos: double (nullable = true)
 |-- sentiment_neu: double (nullable = true)
 |-- sentiment_neg: double (nullable = true)
 |-- sentiment_compound: double (nullable = true)
 |-- tfidf_features: vectorudt (nullable = true)

